# 第二章 `TensorFlow` 基础

## 2-1 TensorFlow基本概念

### 2-1-1 认识 `TensorFlow`

- 使用图 `graphs` 来表示计算任务
- 在被称之为会话 `Session` 的上下文 `context` 中执行图
- 使用 `tensor` 表示数据
- 通过变量 `Variable` 维护状态
- 使用 `feed` 和 `fetch` 可以为任意的操作赋值或者从其中获取数据

`TensorFlow` 是一个变成系统, 使用图 `Graphs` 来表示计算任务, 图 `Graphs` 中的节点称之为 `op`(`operation`), 一个 `op` 获得 `0` 个或者多个 `Tensor`, 执行计算, 产生 `0` 个或者多个 `Tensor`. `Tensor` 看做是一个 `n` 维的数组或列表, 图必须在回话(`Session`)里被启动.

### 2-1-2 `TensorFlow` 的结构

![](picture/2-1-2-1.png)

### 2-1-3 创建图-启动图

In [12]:
import tensorflow as tf

In [13]:
# 创建常量 op, m1与m2
m1 = tf.constant([[3, 3]])
m2 = tf.constant([[2], [3]])
print(m1)
print(m2)

Tensor("Const_6:0", shape=(1, 2), dtype=int32)
Tensor("Const_7:0", shape=(2, 1), dtype=int32)


In [14]:
# 创建一个矩阵乘法op, 把m1和m2传入, 得到一个Tensor
product = tf.matmul(m1, m2)
print(product)

Tensor("MatMul_2:0", shape=(1, 1), dtype=int32)


**方法一: 直接定义会话**

In [15]:
# 定义一个会话, 启动默认图
sess = tf.Session()
# 调用sess中的run方法来执行矩阵乘法op
# run 方法触发了图中的三个op
result = sess.run(product)
print(result)
# 关闭会话
sess.close()

[[15]]


**方法二: 使用 `with...as...` 语句定义会话**

In [16]:
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[15]]


## 2-2 `TensorFlow` 变量的使用

In [17]:
import tensorflow as tf

In [21]:
x = tf.Variable([1, 2])  # 定义一个变量 x
a = tf.constant([3, 3])  # 定义一个常量 a
print(x)
print(a)

<tf.Variable 'Variable_3:0' shape=(2,) dtype=int32_ref>
Tensor("Const_8:0", shape=(2,), dtype=int32)


In [26]:
# 增加一个减法 op
sub = tf.subtract(x, a)
print(sub)

# 增加一个加法 op
add = tf.add(x, a)
print(add)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print("x - a =", sess.run(sub))
    print("x + a =", sess.run(add))

Tensor("Sub_4:0", shape=(2,), dtype=int32)
Tensor("Add_4:0", shape=(2,), dtype=int32)
x - a = [-2 -1]
x + a = [4 5]


**`TensorFlow` 简单使用**

In [32]:
# 创建一个变量, 初始化为 0, name="counter"
state = tf.Variable(0, name="counter")
# 创建一个 op, 作用是使用 state 加1 
new_value = tf.add(state, 1)
# 赋值 op (将新的 new_value 赋值给 state)
update = tf.assign(state, new_value)
# 变量初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(state))
    for _ in range(5):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4
5


## 2-3 `Fetch and Feed`

`Fetch` 在会话中可以执行多个 `op`, 等到它运行的结果

In [33]:
# 定义三个常量
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)

In [34]:
add = tf.add(input2, input3)
mul = tf.multiply(input1, add)

with tf.Session() as sess:
    result = sess.run([mul, add])
    print(result)

[21.0, 7.0]


`Feed` 的数据以字典的形式传入

In [37]:
# 创建占位符
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

# 定义一个乘法 op, 但是并不知道input1与input2的值
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7.0], input2:[2.0]}))

[14.]


## 2-4 `TensorFlow` 简单实用案例

In [59]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [83]:
# 使用 numpy 生成100个随机点
x_data = np.random.rand(100)
y_data = x_data * 0.1 + 0.2 

# 构造一个线性模型
b = tf.Variable(0.0)  # 截距
k = tf.Variable(0.0)  # 斜率
y = k * x_data + b

# 二次代价函数
loss = tf.reduce_mean(tf.square(y_data -y))
# 定义一个梯度下降法来进行训练的优化器
optimizer = tf.train.GradientDescentOptimizer(0.2)
# 最小化代价函数
train = optimizer.minimize(loss)
# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for step in range(201):
        sess.run(train)
        if step % 20 == 0:
            print(step, sess.run([k, b]))

0 [0.050356485, 0.09896989]
20 [0.1008236, 0.19958039]
40 [0.100471094, 0.19976005]
60 [0.10026942, 0.19986278]
80 [0.1001541, 0.1999215]
100 [0.100088134, 0.1999551]
120 [0.10005041, 0.19997433]
140 [0.100028835, 0.19998531]
160 [0.10001649, 0.1999916]
180 [0.10000944, 0.19999519]
200 [0.1000054, 0.19999725]
